In [30]:
from LDA_XGB.pipeline import CopathologyPipeline
from LDA_XGB.data_processor import *
from LDA_XGB.visualizer import CopathologyVisualizer
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
# Optional: pip install statannotations if not installed
from statannotations.Annotator import Annotator

path_all_dx_k_18 = 'C:/Users/WooSikKim/Desktop/Research/projects/co_pathology/scripts/stage_copath/STAGE_CoPathology/LDA_XGB/models/wsev_smc_all_dx_with_cn_k_18.pkl'
mdl_all_dx_k_18 = CopathologyPipeline.load(path_all_dx_k_18)
print(mdl_all_dx_k_18._region_cols)

Pipeline loaded from C:/Users/WooSikKim/Desktop/Research/projects/co_pathology/scripts/stage_copath/STAGE_CoPathology/LDA_XGB/models/wsev_smc_all_dx_with_cn_k_18.pkl
['VA/2', 'VA/4', 'VA/5', 'VA/7', 'VA/8', 'VA/10', 'VA/11', 'VA/12', 'VA/13', 'VA/14', 'VA/15', 'VA/16', 'VA/17', 'VA/18', 'VA/24', 'VA/26', 'VA/28', 'VA/31', 'VA/41', 'VA/43', 'VA/44', 'VA/46', 'VA/47', 'VA/49', 'VA/50', 'VA/51', 'VA/52', 'VA/53', 'VA/54', 'VA/58', 'VA/60', 'VA/63', 'VA/77', 'VA/1002', 'VA/1003', 'VA/1005', 'VA/1006', 'VA/1007', 'VA/1008', 'VA/1009', 'VA/1010', 'VA/1011', 'VA/1012', 'VA/1013', 'VA/1014', 'VA/1015', 'VA/1016', 'VA/1017', 'VA/1018', 'VA/1019', 'VA/1020', 'VA/1021', 'VA/1022', 'VA/1023', 'VA/1024', 'VA/1025', 'VA/1026', 'VA/1027', 'VA/1028', 'VA/1029', 'VA/1030', 'VA/1031', 'VA/1034', 'VA/1035', 'VA/2002', 'VA/2003', 'VA/2005', 'VA/2006', 'VA/2007', 'VA/2008', 'VA/2009', 'VA/2010', 'VA/2011', 'VA/2012', 'VA/2013', 'VA/2014', 'VA/2015', 'VA/2016', 'VA/2017', 'VA/2018', 'VA/2019', 'VA/2020', 'V

In [31]:
nacc_data_path = 'C:/Users/WooSikKim/Desktop/Research/projects/co_pathology/scripts/stage_copath/data/nacc'
raw_nacc_df = pd.read_csv(os.path.join(nacc_data_path, 'NACC_external_260212_all_dx_lda_k_18.csv'))
raw_nacc_df['copath'] = raw_nacc_df['copath'].replace('NC+PD','PD')
# raw_nacc_df = raw_nacc_df[~raw_nacc_df['copath'].str.contains('NC', na=False)]
raw_nacc_df = raw_nacc_df[
    ~raw_nacc_df['copath'].str.contains('NC|MCI|iMCI', na=False)
]
raw_nacc_df.dropna(subset='copath', inplace=True)

lda_dx_topic_mean_df = pd.read_csv('C:/Users/WooSikKim/Desktop/Research/projects/co_pathology/scripts/stage_copath/results/wsev_smc_all_dx/18/lda_diagnosis_topic_expression.csv')
lda_dx_topic_mean_df = lda_dx_topic_mean_df[lda_dx_topic_mean_df['DX']!='NC']
lda_dx_topic_mean_df['DX'] = lda_dx_topic_mean_df['DX'].replace('DLB', 'LBD')

print(raw_nacc_df['copath'].value_counts())
topic_cols = [f'Topic_{i}' for i in range(18)]


copath
AD                334
AD+VD              50
FTD_ANY            40
LBD                24
AD+PCA             22
PD                 20
AD+LBD             17
VD                  9
AD+FTD_ANY          8
PD+LBD              4
LBD+VD              3
VD+FTD_ANY          3
LBD+VD+SVAD         3
VD+SVAD             2
AD+VD+SVAD          2
AD+LBD+PCA          2
AD+LBD+VD           1
AD+PD+LBD           1
AD+VD+FTD_ANY       1
LBD+FTD_ANY         1
AD+LBD+FTD_ANY      1
AD+PD               1
AD+VD+PCA           1
AD+SVAD             1
Name: count, dtype: int64


In [39]:
# raw_nacc_df['copath'].value_counts().to_csv('copath_count.csv')
print(raw_nacc_df.shape)

(551, 277)


In [ ]:
for dx in lda_dx_topic_mean_df['DX']:
    mean_vec = lda_dx_topic_mean_df.loc[lda_dx_topic_mean_df['DX'] == dx, topic_cols].values.flatten()
    raw_nacc_df[f'ND_{dx}'] = raw_nacc_df[topic_cols].values @ mean_vec
    
copath_dfs = {grp: subdf.copy() for grp, subdf in raw_nacc_df.groupby('copath')}
nd_cols = [col for col in raw_nacc_df.columns if col.startswith('ND_')]

# for grp, subdf in copath_dfs.items():
#     melted = subdf.melt(
#         value_vars=nd_cols,
#         var_name='Diagnosis',
#         value_name='ND_score'
#     )
    
#     plt.figure(figsize=(8,5))
#     sns.boxplot(data=melted, x='Diagnosis', y='ND_score')
#     plt.title(f'ND score by Diagnosis\nCoPath: {grp}')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()

In [ ]:
from statannotations.Annotator import Annotator

# ---------------------------
# Categorize copath groups
# ---------------------------
def count_pathologies(label):
    return len(label.split('+'))

print(raw_nacc_df.shape)
raw_nacc_df['num_path'] = raw_nacc_df['copath'].apply(count_pathologies)

# manual ordering by pathology count
singular = [g for g in copath_dfs if raw_nacc_df.loc[raw_nacc_df['copath']==g, 'num_path'].iloc[0]==1]
double = [g for g in copath_dfs if raw_nacc_df.loc[raw_nacc_df['copath']==g, 'num_path'].iloc[0]==2]
more_than_two = [g for g in copath_dfs if raw_nacc_df.loc[raw_nacc_df['copath']==g, 'num_path'].iloc[0]>2]

# Combine for plotting order
ordered_groups = singular + double + more_than_two

# ---------------------------
# Plot subplots
# ---------------------------
num_subgroups = len(ordered_groups)
cols = 2
rows = int(np.ceil(num_subgroups / cols))
fig, axes = plt.subplots(rows, cols, figsize=(12, 5*rows))
axes = axes.flatten()

box_colors = sns.color_palette("Set2", n_colors=len(nd_cols))

for i, grp in enumerate(ordered_groups):
    subdf = copath_dfs[grp]
    melted = subdf.melt(
        value_vars=nd_cols,
        var_name='Diagnosis',
        value_name='ND_score'
    )
    
    ax = axes[i]
    sns.boxplot(
        data=melted,
        x='Diagnosis',
        y='ND_score',
        palette=box_colors,
        ax=ax
    )
    
    ax.set_title(f"{grp} (n={len(subdf)})", fontsize=12)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    
    # t-test only, hide non-significant
    try:
        pairs = [(a, b) for idx, a in enumerate(nd_cols) for b in nd_cols[idx+1:]]
        annotator = Annotator(ax, pairs, data=melted, x='Diagnosis', y='ND_score')
        annotator.configure(
            test='t-test_ind',
            text_format='star',
            loc='inside',
            hide_non_significant=True,
            verbose=0,
            comparisons_correction='BH'
        )
        annotator.apply_and_annotate()
    except Exception as e:
        print(f"Statannotations skipped for {grp}: {e}")

# Remove unused axes
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
from statannotations.Annotator import Annotator
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticker

# ---------------------------
# Filter out subgroups with only 1 subject
# ---------------------------
filtered_copath_dfs = {grp: subdf for grp, subdf in copath_dfs.items() if len(subdf) > 1}

num_subgroups = len(filtered_copath_dfs)
cols = 2
rows = int(np.ceil(num_subgroups / cols))
fig, axes = plt.subplots(rows, cols, figsize=(12, 5*rows))
axes = axes.flatten()

# Common y-limits across all remaining ND scores
# 1️⃣ Compute common y-limits BEFORE the loop
y_min = raw_nacc_df[nd_cols].min().min() * 0.95
y_max = raw_nacc_df[nd_cols].max().max() * 1.05

# 2️⃣ Start plotting subplots
for i, (grp, subdf) in enumerate(filtered_copath_dfs.items()):
    
    # Melt ND scores for plotting
    melted = subdf.melt(
        value_vars=nd_cols,
        var_name='Diagnosis',
        value_name='ND_score'
    )
    
    ax = axes[i]
    
    # --- Boxplot ---
    sns.boxplot(
        data=melted,
        x='Diagnosis',
        y='ND_score',
        palette=box_colors,
        ax=ax
    )
    
    # --- Quick edit: set same y-limits for all subplots ---
    # ax.set_ylim(y_min, y_max)
    
    # Title with subject count
    ax.set_title(f"{grp} (n={len(subdf)})", fontsize=12)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

    # after plotting the boxplot
    ax.yaxis.set_major_locator(mticker.MultipleLocator(0.05))  # grid every 0.1 units
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # --- t-test annotations (only significant) ---
    try:
        # pairs = [(a, b) for idx, a in enumerate(nd_cols) for b in nd_cols[idx+1:]]
        pairs = []
        for i_dx, dx1 in enumerate(nd_cols[:-1]):
            for dx2 in nd_cols[i_dx+1:]:
                pairs.append((dx1, dx2))  # draw comparisons sequentially

        annotator = Annotator(ax, pairs, data=melted, x='Diagnosis', y='ND_score')
        annotator.configure(
            test='t-test_ind',
            text_format='star',
            loc='inside',
            hide_non_significant=True,
            verbose=0,
            comparisons_correction='bonferroni',
            line_width=0.5
        )
        annotator.apply_and_annotate()
    except Exception as e:
        print(f"Statannotations skipped for {grp}: {e}")
        
    ax.set_ylim(y_min, 0.65)

# Remove unused axes
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
## PD Pathologies ##
nacc_data_path = 'C:/Users/WooSikKim/Desktop/Research/projects/co_pathology/scripts/stage_copath/data/nacc'
pd_raw_df = pd.read_csv(os.path.join(nacc_data_path, 'NACC_external_260212_all_dx_lda_k_18.csv'))
pd_raw_df = pd_raw_df[
    pd_raw_df['copath'].str.contains('PD', na=False)
]
print(pd_raw_df['copath'].value_counts())